In [1]:
from flurs.datasets import fetch_movielens
from flurs.recommender import BRISMFRecommender
from flurs.evaluator import Evaluator

import logging
import os
import sys

ML_PATH = 'C:\\recsys\\datasets\\movielens\\ml-1m\\'

program = os.path.basename(ML_PATH)
logger = logging.getLogger(program)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
logging.root.setLevel(level=logging.INFO)
logger.info('running %s' % ' '.join(sys.argv))

logging.info('converting data into FluRS input object')
data = fetch_movielens(data_home=ML_PATH, size='1m')

2019-02-14 17:22:12,394 : INFO : running C:\Users\pinguim\Anaconda3\envs\recommender\lib\site-packages\ipykernel_launcher.py -f C:\Users\pinguim\AppData\Roaming\jupyter\runtime\kernel-40badba0-b9a6-4dca-ad0f-5ce07baf6224.json
2019-02-14 17:22:12,398 : INFO : converting data into FluRS input object


In [2]:
logging.info('initialize recommendation model and evaluation module')
rec = BRISMFRecommender()
rec.initialize()
evaluator = Evaluator(rec, data.can_repeat)

n_batch_train = int(data.n_sample * 0.2)  # 20% for pre-training to avoid cold-start
n_batch_test = int(data.n_sample * 0.1)  # 10% for evaluation of pre-training
batch_tail = n_batch_train + n_batch_test

# pre-train
# 20% for batch training | 10% for batch evaluate
# after the batch training, 10% samples are used for incremental updating
logging.info('batch pre-training before streaming input')
evaluator.fit(
    data.samples[:n_batch_train],
    data.samples[n_batch_train:batch_tail],
    n_epoch=1  # single pass even for batch training
)

# 70% incremental evaluation and updating
logging.info('incrementally predict, evaluate and update the recommender')
res = evaluator.evaluate(data.samples[batch_tail:])

2019-02-14 17:22:19,576 : INFO : initialize recommendation model and evaluation module
2019-02-14 17:22:19,587 : INFO : batch pre-training before streaming input
2019-02-14 17:22:35,682 : INFO : incrementally predict, evaluate and update the recommender


<generator object Evaluator.evaluate at 0x000000FEA99E28E0>


In [4]:
for i in res:
    print(i)

(0.024491401126956913, 17, 0.001845325382397178, 0.00014112569552516066)
(0.022962222014087708, 31, 0.0006222360211791192, 4.148240141194082e-05)
(0.021049658826973294, 200, 0.0008052717717390231, 6.329273617491915e-05)
(0.005062492021540632, 660, 0.0008240885311423807, 6.28650825521132e-05)
(0.017734621686385954, 335, 0.000885670652826083, 7.227346225379544e-05)
(0.010398808303917373, 48, 0.0008317862963528391, 6.585865791174095e-05)
(0.0017065093306360968, 402, 0.0008078376934758449, 6.457569704333005e-05)
(9.599214418010149e-05, 161, 0.0016391963362058781, 0.00015566591870047608)
(0.000674196927569426, 405, 0.0008886642281857177, 5.388435647322998e-05)
(0.009023188023829398, 236, 0.0007971463529057449, 6.243742892930726e-05)
(0.0016180070225354504, 2109, 0.0007642170239498813, 6.714161878014491e-05)
(0.003437704873401337, 1888, 0.0008856706528260899, 6.243742892932114e-05)
(0.007760978426636589, 1469, 0.000755236297871012, 5.645027821006565e-05)
(0.012557352261980181, 52, 0.00073599

KeyboardInterrupt: 